In [18]:
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
import numpy as np

In [19]:
current_time = datetime.datetime.today()
year = str(current_time.year)
month = str(current_time.month)
day = str(current_time.day)

if len(month) == 1:
	month = "0" + month
if len(day) == 1:
	day = "0" + day

split_date = year + '-' + month + '-' + day
split_date

split_date = '2023-04-01'

In [20]:
# matches = pd.read_csv("../data/premier_league_2019-2023.csv", index_col=0)
li = []

# li.append(pd.read_csv("../data/premier_league_2019-2023.csv", index_col=0))
li.append(pd.read_csv("../data/la_liga_2019-2023.csv", index_col=0))
# li.append(pd.read_csv("../data/bundesliga_2019-2023.csv", index_col=0))
# li.append(pd.read_csv("../data/serie_a_2019-2023.csv", index_col=0))
matches = pd.concat(li, axis=0, ignore_index=True)
matches.shape

(3478, 129)

In [21]:
matches["target"] = (matches["result"] == "W").astype("int")

In [22]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes == 1
matches["opp_code"] = matches["opponent"].astype("category").cat.codes #each opponent now has their own code

In [23]:
matches["date"] = pd.to_datetime(matches["date"])
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int") #remove : and minutes, keep just hour
matches["day_code"] = matches["date"].dt.dayofweek

In [24]:
matches["formation_code"] = matches["formation"].astype("category").cat.codes #each opponent now has their own code
matches["referee_code"] = matches["referee"].astype("category").cat.codes #each opponent now has their own code

In [25]:
rf = RandomForestClassifier(n_estimators=10000, min_samples_split=4, random_state=1, max_depth=3)

In [26]:
train = matches[matches["date"] < split_date]
test = matches[matches["date"] > split_date]

In [27]:
predictors = ["venue_code", "opp_code", "hour", "day_code", "formation_code", "referee_code"]

In [28]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date") # date wird immer spaeter von 2019 bis 2024
    weights = [0.3333333, 0.333333, 0.333333] # Define your own weights here
    rolling_stats = group[cols].rolling(3, closed='left').apply(lambda x: (x * weights).sum())
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [29]:
grouped_matches = matches.groupby("team")

In [30]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "sota", "saves", "opp", "stp"]
# , "blocks", "pass", "int", "err", "cmp", "att", "totdist", "prgdist", "sca", "passlive", "passdead", "tkl", "tklw", "crdy", "crdr", "2crdy", "fls", "fld", "off", "crs", "recov", "won"
# funkt in premier aber nicht la liga: tackles und tackles won
# "blocks", "pass", "int", "err"
# "cmp", "att", "totdist", "prgdist"
# "sca", "passlive", "passdead"
new_cols = [f"{c}_rolling" for c in cols]

In [31]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

/var/folders/yn/th9j1n395vq70h3nx8132vg00000gn/T/ipykernel_42587/110184434.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


,date,time,round,day,venue,result,gf,ga,opponent,xg,...,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,sota_rolling,saves_rolling,opp_rolling,stp_rolling
573,2019-09-15,14:00,Matchweek 4,Sun,Home,L,0.0,1.0,Sevilla,0.8,...,7.999995,1.666666,15.099990,0.333333,0.000000,0.000000,1.999999,1.666666,14.999992,0.000000
574,2019-09-22,18:30,Matchweek 5,Sun,Away,L,0.0,2.0,Athletic Club,0.2,...,7.333329,1.000000,14.733324,0.000000,0.000000,0.000000,2.999997,2.333331,12.333324,0.000000
575,2019-09-26,21:00,Matchweek 6,Thu,Away,L,0.0,3.0,Real Sociedad,0.3,...,5.333329,0.666666,15.433320,0.000000,0.000000,0.000000,3.666663,2.333331,12.666658,0.000000
576,2019-09-29,16:00,Matchweek 7,Sun,Home,W,2.0,0.0,Mallorca,2.1,...,5.333330,0.333333,16.899988,0.000000,0.000000,0.000000,5.666663,3.333332,13.333325,0.000000
577,2019-10-05,18:30,Matchweek 8,Sat,Away,L,1.0,2.0,Valencia,1.7,...,6.333328,1.999998,18.699987,0.000000,0.333333,0.333333,3.666664,1.666665,10.666659,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,2023-12-20,21:30,Matchweek 18,Wed,Home,W,3.0,2.0,Celta Vigo,2.4,...,9.333326,2.999998,16.599988,0.000000,0.000000,0.000000,5.999995,3.333331,24.333323,0.666666
3342,2024-01-02,21:30,Matchweek 19,Tue,Away,L,1.0,3.0,Valencia,0.5,...,11.333328,3.999997,15.899989,0.000000,0.333333,0.333333,5.999996,2.999998,15.999988,0.999999
3343,2024-01-13,13:00,Matchweek 20,Sat,Away,L,0.0,3.0,Las Palmas,2.0,...,7.333327,3.999996,15.533323,0.000000,0.333333,0.333333,6.999996,3.999998,14.999989,1.333332
3344,2024-01-20,16:15,Matchweek 21,Sat,Home,D,1.0,1.0,Mallorca,1.9,...,11.999991,4.666664,14.766656,0.666666,0.333333,0.333333,4.999996,2.333332,13.666660,1.000000


In [32]:
matches_rolling.index = range(matches_rolling.shape[0])

In [33]:
def make_predictions(data, predictors):
    train = data[data["date"] < split_date]
    test = data[data["date"] > split_date]

    rf.fit(train[predictors], train["target"])

    predsWinsTrain = rf.predict(train[predictors])
    predsWinsTest = rf.predict(test[predictors])

    probabilities = rf.predict_proba(test[predictors])
    probabilities = probabilities[:, 0]

    combinedWinsTrain = pd.DataFrame(dict(actual=train["target"], predicted=predsWinsTrain), index=train.index)
    errorWinsTrain = precision_score(train["target"], predsWinsTrain)

    combinedWinsTest = pd.DataFrame(dict(actual=test["target"], predicted=predsWinsTest), index=test.index)
    combinedWinsTest['probabilities'] = probabilities
    
    errorWinsTest = precision_score(test["target"], predsWinsTest)

    print(errorWinsTrain)
    print(errorWinsTest)

    return combinedWinsTest, errorWinsTest

In [34]:
combinedWins, errorWins = make_predictions(matches_rolling, predictors + new_cols)

0.8229166666666666
0.6829268292682927


In [35]:
combinedWins = combinedWins.merge(matches_rolling[["result", "gf", "ga", "team", "opponent", "venue", "date", "hour", "formation_code", "referee_code"]], left_index=True, right_index=True)

In [36]:
finalGuess = combinedWins[(combinedWins["predicted"] == 1) & (combinedWins["venue"] == "Home")].sort_values("date")
finalGuess

,actual,predicted,probabilities,result,gf,ga,team,opponent,venue,date,hour,formation_code,referee_code
2567,0,1,0.447077,L,2.0,3.0,Real Madrid,Villarreal,Home,2023-04-08,21,16,1
2569,1,1,0.416429,W,2.0,0.0,Real Madrid,Celta Vigo,Home,2023-04-22,21,16,4
3361,1,1,0.458316,W,4.0,2.0,Villarreal,Espanyol,Home,2023-04-27,19,12,7
2571,1,1,0.412599,W,4.0,2.0,Real Madrid,Almería,Home,2023-04-29,18,16,11
3362,1,1,0.445815,W,3.0,1.0,Villarreal,Celta Vigo,Home,2023-04-30,16,12,25
3364,1,1,0.483628,W,5.0,1.0,Villarreal,Athletic Club,Home,2023-05-13,18,16,6
677,0,1,0.457463,L,1.0,2.0,Barcelona,Real Sociedad,Home,2023-05-20,21,16,1
2912,0,1,0.467129,D,0.0,0.0,Sevilla,Betis,Home,2023-05-21,21,12,15
682,1,1,0.492765,W,2.0,0.0,Barcelona,Cádiz,Home,2023-08-20,19,4,3
2581,1,1,0.490638,W,2.0,1.0,Real Madrid,Getafe,Home,2023-09-02,16,14,22


In [37]:
print(len(finalGuess))
print(len(finalGuess[finalGuess["actual"] == 1]))
print(int(len(finalGuess[finalGuess["actual"] == 1]) / len(finalGuess) * 10000) / 100, "%")
num = len(finalGuess[finalGuess["actual"] == 1]) / len(finalGuess)
print("Minimum win:", 1 + (1.0 - num) / num)
print("Analyzed games:", len(matches_rolling))

32
23
71.87 %
Minimum win: 1.391304347826087
Analyzed games: 3391


In [38]:
print(len(combinedWins[combinedWins["actual"] == combinedWins["predicted"]]) / len(combinedWins))
print(len(combinedWins))

0.6526315789473685
665
